In [1]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import urllib.request
import datetime
engine = create_engine('sqlite:///taiwanstock.db')

## Download monthly world commodities excel from World Bank

In [2]:
commodities_url = 'http://pubdocs.worldbank.org/en/561011486076393416/CMO-Historical-Data-Monthly.xlsx'
misc_dir = 'misc/'
saveFile = 'commodities_monthly_raw.xlsx'
save_dir = misc_dir + saveFile
urllib.request.urlretrieve(commodities_url, save_dir)

('misc/commodities_monthly_raw.xlsx',
 <http.client.HTTPMessage at 0x206d13b5f28>)

## Read data from excel to Pandas dataframe
* Change the index to number
* modify the original date format to python datetime formate

In [3]:
commodities = pd.read_excel('misc/commodities_monthly_raw.xlsx'\
                            , sheetname='Monthly Prices'\
                           , header=6, index_col=None)
commodities=commodities.replace(to_replace='..', value=np.nan)
commodities['date']=commodities.index
commodities.index = range(len(commodities))
for i, data in enumerate(commodities['date']):
    date = data.replace('M','')
    date = datetime.datetime.strptime(date, "%Y%m").date()
    commodities.loc[i, 'date'] = date

## Arrange the table to new dataframe for storing into database

In [4]:
#df = pd.DataFrame(columns=['date', 'commodity', 'price'])
columns = ['date', 'commodity', 'price']
dataList = list()
cols = commodities.columns.tolist()
for commodity in cols:
    if commodity == 'date':
        break
    else:
        for i, date in enumerate(commodities['date']):
            price = commodities.loc[i, commodity]
            tempList = [date, commodity, price]
            dataList.append(tempList)
npData = np.asarray(dataList)
df = pd.DataFrame(columns=['date', 'commodity', 'price'], data=npData)

## Store data into database

In [5]:
df.to_sql('worldcommodities',con=engine,if_exists='replace')